In [4]:
import numpy as np
import pandas as pd
import re
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split

In [5]:
dataset = pd.read_csv('Sentiment.csv')

In [6]:
dataset.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,name,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,I_Am_Kenzi,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,PeacefulQuest,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,PussssyCroook,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,MattFromTexas31,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,sharonDay5,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [7]:
dataset = dataset[['text','sentiment']]
dataset = dataset[dataset.sentiment != "Neutral"]

In [8]:
dataset.head()

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative


In [9]:
dataset.shape

(10729, 2)

In [10]:
dataset['text'] = dataset['text'].apply(lambda x: x.lower())
dataset['text'] = dataset['text'].apply(lambda x: re.sub(r'[\W_]+', ' ', x))
dataset['text'] = dataset['text'].apply(lambda x: re.sub('rt','',x))
dataset['text'] = dataset['text'].apply(lambda x: x.lstrip())

In [11]:
dataset['sentiment'] = dataset['sentiment'].apply(lambda x: re.sub('Positive','1', x))
dataset['sentiment'] = dataset['sentiment'].apply(lambda x: re.sub('Negative','0', x))

In [12]:
dataset.head()

,text,sentiment
1,scottwalker didn t catch the full gopdebate la...,1
3,robgeorge that carly fiorina is trending hours...,1
4,danscavino gopdebate w realdonaldtrump deliver...,1
5,gregabbott tx tedcruz on my first day i will r...,1
6,warriorwoman91 i liked her and was happy when ...,0


In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [29]:
max_words = 2000

In [30]:
cv = CountVectorizer(max_features = max_words ,analyzer = 'word', ngram_range=(1, 1))

In [31]:
cv.fit(dataset['text'].values)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=2000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [32]:
cv.vocabulary_

{'scottwalker': 1541,
 'didn': 487,
 'the': 1753,
 'full': 721,
 'gopdebate': 765,
 'last': 1004,
 'night': 1200,
 'here': 827,
 'are': 115,
 'some': 1625,
 'of': 1234,
 'scott': 1540,
 'best': 190,
 'lines': 1043,
 'in': 902,
 '90': 23,
 'seconds': 1546,
 'walker16': 1894,
 'http': 867,
 'co': 371,
 'that': 1751,
 'carly': 307,
 'fiorina': 675,
 'is': 931,
 'hours': 860,
 'after': 49,
 'her': 826,
 'debate': 450,
 'any': 104,
 'men': 1117,
 'just': 969,
 'says': 1533,
 'she': 1583,
 'on': 1250,
 'danscavino': 438,
 'realdonaldtrump': 1438,
 'delivered': 468,
 'highest': 832,
 'ratings': 1430,
 'history': 843,
 'presidential': 1367,
 'debates': 452,
 'trump2016': 1820,
 'tedcruz': 1732,
 'my': 1174,
 'first': 678,
 'day': 442,
 'will': 1938,
 'every': 596,
 'illegal': 889,
 'executive': 609,
 'action': 37,
 'taken': 1714,
 'by': 277,
 'obama': 1228,
 'foxnews': 703,
 'liked': 1038,
 'and': 89,
 'was': 1905,
 'happy': 803,
 'when': 1924,
 'heard': 819,
 'going': 750,
 'to': 1789,
 'be':

In [33]:
x = cv.transform(dataset['text'].values).toarray()

In [34]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [35]:
tokenizer = Tokenizer(num_words = max_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', split=' ')

In [36]:
tokenizer.fit_on_texts(dataset['text'].values)

In [37]:
x_tokenizer = tokenizer.texts_to_sequences(dataset['text'].values)

In [38]:
x_tokenizer = pad_sequences(x_tokenizer)

In [39]:
x_tokenizer

array([[   0,    0,    0, ...,   13,    4,   11],
       [   0,    0,    0, ...,  154,   14,   22],
       [   0,    0,    0, ...,   13,    4,   11],
       ...,
       [   0,    0,    0, ...,   77,   71,    3],
       [   0,    0,    0, ..., 1015, 1320,   79],
       [   0,    0,    0, ...,    4,   11,  716]])

In [40]:
y = np.array(dataset['sentiment'])

In [41]:
y

array(['1', '1', '1', ..., '1', '0', '1'], dtype=object)

# Data preprocessing is complete

In [42]:
x_train, x_test, y_train, y_test = train_test_split(x_tokenizer, y, test_size = 0.33, random_state = 0)

In [43]:
x_train.shape

(7188, 30)

# Dataset has been split in training set and test set

# Time to build a NN using RNN and LSTM

In [44]:
model = Sequential()
model.add(Embedding(input_dim = max_words, output_dim = 50, embeddings_initializer = 'uniform',input_length = x_train.shape[1]))

Instructions for updating:
Colocations handled automatically by placer.


In [45]:
model.add(LSTM(units = 50, return_sequences=False))
model.add(Dropout(rate = 0.2))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [46]:
model.add(Dense(units = 1, activation = 'relu'))

In [47]:
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            100000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 120,251
Trainable params: 120,251
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.fit(x_train, y_train, epochs = 10, batch_size = 500)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
7188/7188 [==============================] - 2s 321us/step - loss: 0.6622 - acc: 0.7940
Epoch 2/10
7188/7188 [==============================] - 1s 106us/step - loss: 0.4891 - acc: 0.7940
Epoch 3/10
7188/7188 [==============================] - 1s 104us/step - loss: 0.4576 - acc: 0.7940
Epoch 4/10
7188/7188 [==============================] - 1s 107us/step - loss: 0.4119 - acc: 0.7977
Epoch 5/10
7188/7188 [==============================] - 1s 112us/step - loss: 0.3546 - acc: 0.8376
Epoch 6/10
7188/7188 [==============================] - 1s 109us/step - loss: 0.3111 - acc: 0.8535
Epoch 7/10
7188/7188 [==============================] - 1s 106us/step - loss: 0.2891 - acc: 0.8614
Epoch 8/10
7188/7188 [==============================] - 1s 107us/step - loss: 0.2742 - acc: 0.8666
Epoch 9/10
7188/7188 [==============================] - 1s 109us/step - loss: 0.2675 - acc: 0.8602
Epoch 10/10
7188/7188 [==============================] - 1s 1